Default as given

In [8]:
from ibapi.client import *
from ibapi.common import ListOfHistoricalTickLast
from ibapi.wrapper import *
import pandas as pd
dict_list = []
class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: int):
        mycontract = Contract()
        mycontract.symbol = 'AAPL'
        mycontract.secType = 'STK'
        mycontract.exchange = 'SMART'
        mycontract.currency = 'USD'

        self.reqHistoricalTicks(
            reqId=orderId,
            contract=mycontract,
            startDateTime='20231204 09:30:00 US/Eastern',
            endDateTime='',
            numberOfTicks=1000,
            whatToShow='TRADES',
            useRth=1,
            ignoreSize=True,
            miscOptions= []
        )
    
    

    def historicalTicksLast(self, reqId: int, ticks: ListOfHistoricalTickLast, done: bool):
        global dict_list 
        for tick in ticks:
            dict_new = {}
            dict_new['reqId'] = reqId
            dict_new['Time'] = tick.time
            dict_new['TickAttribLast'] = tick.tickAttribLast
            dict_new['Price'] = tick.price
            dict_new['Size'] = tick.size
            dict_new['Exchange'] = tick.exchange
            dict_new['SpecialConditions'] = tick.specialConditions
            dict_list.append(dict_new)
        if done:
            self.disconnect()

    def error(self, reqId: TickerId, errorCode: int, errorString: str, advancedOrderRejectJson=""):
        print(reqId, errorCode, errorString, advancedOrderRejectJson)
    

app = TestApp()
app.connect("127.0.0.1", 7497, 111)
app.run()

In [9]:
df = pd.DataFrame(dict_list)
df

""


Code for Trades

In [1]:
from ibapi.client import EClient
from ibapi.common import ListOfHistoricalTickLast
from ibapi.contract import Contract
from ibapi.wrapper import EWrapper
import pandas as pd
from datetime import datetime, timedelta
import pytz

class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.last_time = None
        self.start_time = '20231205 09:30:00 US/Eastern'

    def nextValidId(self, orderId: int):
        self.fetchData(orderId)

    def fetchData(self, orderId):
        mycontract = Contract()
        mycontract.symbol = 'AAPL'
        mycontract.secType = 'STK'
        mycontract.exchange = 'SMART'
        mycontract.currency = 'USD'

        self.reqHistoricalTicks(
            reqId=orderId,
            contract=mycontract,
            startDateTime=self.start_time,
            endDateTime='',
            numberOfTicks=1000,
            whatToShow='TRADES',
            useRth=1,
            ignoreSize=True,
            miscOptions=[]
        )

    def historicalTicksLast(self, reqId: int, ticks: ListOfHistoricalTickLast, done: bool):
        for tick in ticks:
            self.data.append({
                'reqId': reqId,
                'Time': tick.time,
                'TickAttribLast': tick.tickAttribLast,
                'Price': tick.price,
                'Size': tick.size,
                'Exchange': tick.exchange,
                'SpecialConditions': tick.specialConditions
            })
            self.last_time = tick.time
            timezone = pytz.timezone("US/Eastern")
            if(datetime.fromtimestamp(tick.time, tz=timezone).strftime('%Y%m%d %H:%M:%S US/Eastern')==('20231205 09:30:03 US/Eastern')): #end_time
                print("Reached end time")
                self.disconnect()
                return # this ensures remaining ticks are not checked/called.
            
        if done:
            timezone = pytz.timezone("US/Eastern")
            market_close = timezone.localize(datetime.strptime('20231205 16:00:00', '%Y%m%d %H:%M:%S'))
            last_tick_time = datetime.fromtimestamp(self.last_time, tz=timezone)
            if last_tick_time < market_close:
                self.start_time = last_tick_time.strftime('%Y%m%d %H:%M:%S US/Eastern')
                self.fetchData(reqId + 1)
                print(self.start_time)
            else:
                self.disconnect()

    def error(self, reqId: int, errorCode: int, errorString: str, advancedOrderRejectJson=""):
        print(reqId, errorCode, errorString, advancedOrderRejectJson)
        if (errorCode==10314):
            self.disconnect()
        

app = TestApp()
app.connect("127.0.0.1", 7497, 111)
app.run()

df = pd.DataFrame(app.data)


-1 2104 Market data farm connection is OK:usfuture 
-1 2104 Market data farm connection is OK:cafarm 
-1 2104 Market data farm connection is OK:eufarmnj 
-1 2104 Market data farm connection is OK:usfarm 
-1 2106 HMDS data farm connection is OK:ushmds 
-1 2158 Sec-def data farm connection is OK:secdefil 
20231205 09:30:02 US/Eastern
Reached end time


In [3]:
df

,reqId,Time,TickAttribLast,Price,Size,Exchange,SpecialConditions
0,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,183,ARCA,FT
1,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,813,ARCA,T
2,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,1000,FINRA,
3,1,1701786600,"PastLimit: 0, Unreported: 1",190.22,1,DRCTEDGE,I
4,1,1701786600,"PastLimit: 0, Unreported: 1",190.22,3,DRCTEDGE,I
...,...,...,...,...,...,...,...
1610,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,1,FINRA,4 I
1611,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,22,FINRA,4 I
1612,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,36,FINRA,4 I
1613,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,1,FINRA,4 I


Code for Bid and Ask

In [7]:
from ibapi.client import EClient
from ibapi.common import ListOfHistoricalTickBidAsk, ListOfHistoricalTickLast
from ibapi.contract import Contract
from ibapi.wrapper import EWrapper
import pandas as pd
from datetime import datetime, timedelta
import pytz

class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.last_time = None
        self.start_time = '20231205 09:30:00 US/Eastern'

    def nextValidId(self, orderId: int):
        self.fetchData(orderId)

    def fetchData(self, orderId):
        mycontract = Contract()
        mycontract.symbol = 'AAPL'
        mycontract.secType = 'STK'
        mycontract.exchange = 'SMART'
        mycontract.currency = 'USD'

        self.reqHistoricalTicks(
            reqId=orderId,
            contract=mycontract,
            startDateTime=self.start_time,
            endDateTime='',
            numberOfTicks=1000,
            whatToShow='BID_ASK',
            useRth=1,
            ignoreSize=True,
            miscOptions=[]
        )

    def historicalTicksBidAsk(self, reqId: int, ticks: ListOfHistoricalTickBidAsk, done: bool):       
        for tick in ticks:
            self.data.append({
                'Time': tick.time,
                'TickAttribLast': tick.tickAttribBidAsk,
                'priceBid': tick.priceBid,
                'priceAsk': tick.priceAsk,
                'sizeBid': tick.sizeBid,
                'sizeAsk': tick.sizeAsk 
            })
            # self.last_time = tick.time
            timezone = pytz.timezone("US/Eastern")
            if(datetime.fromtimestamp(tick.time, tz=timezone).strftime('%Y%m%d %H:%M:%S US/Eastern')==('20231205 09:30:03 US/Eastern')):
                print("reached")
                self.disconnect()
                return # this ensures remaining ticks are not checked/called.

        if done:
            self.disconnect()

    def error(self, reqId: int, errorCode: int, errorString: str, advancedOrderRejectJson=""):
        print(reqId, errorCode, errorString, advancedOrderRejectJson)
        if (errorCode==10314):
            self.disconnect()
        

app = TestApp()
app.connect("127.0.0.1", 7497, 111)
app.run()

df4 = pd.DataFrame(app.data)


-1 2104 Market data farm connection is OK:usfuture 
-1 2104 Market data farm connection is OK:cafarm 
-1 2104 Market data farm connection is OK:eufarmnj 
-1 2104 Market data farm connection is OK:usfarm 
-1 2106 HMDS data farm connection is OK:ushmds 
-1 2158 Sec-def data farm connection is OK:secdefil 
reached


In [8]:
df4

,Time,TickAttribLast,priceBid,priceAsk,sizeBid,sizeAsk
0,1701786599,"BidPastLow: 0, AskPastHigh: 0",190.15,190.21,1600,900
1,1701786600,"BidPastLow: 0, AskPastHigh: 0",190.18,190.30,100,5700
2,1701786600,"BidPastLow: 0, AskPastHigh: 0",190.17,190.23,200,100
3,1701786600,"BidPastLow: 0, AskPastHigh: 0",190.17,190.30,300,5800
4,1701786600,"BidPastLow: 0, AskPastHigh: 0",190.17,190.21,500,200
...,...,...,...,...,...,...
170,1701786602,"BidPastLow: 0, AskPastHigh: 0",190.30,190.32,700,1100
171,1701786602,"BidPastLow: 0, AskPastHigh: 0",190.31,190.32,200,1400
172,1701786602,"BidPastLow: 0, AskPastHigh: 0",190.31,190.33,2500,600
173,1701786602,"BidPastLow: 0, AskPastHigh: 0",190.32,190.33,400,800


In [21]:
merged_df = pd.merge(df, df4, on='Time', how='left')

merged_df

,reqId,Time,TickAttribLast_x,Price,Size,Exchange,SpecialConditions,TickAttribLast_y,priceBid,priceAsk,sizeBid,sizeAsk
0,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,183,ARCA,FT,"BidPastLow: 0, AskPastHigh: 0",190.18,190.30,100,5700
1,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,183,ARCA,FT,"BidPastLow: 0, AskPastHigh: 0",190.17,190.23,200,100
2,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,183,ARCA,FT,"BidPastLow: 0, AskPastHigh: 0",190.17,190.30,300,5800
3,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,183,ARCA,FT,"BidPastLow: 0, AskPastHigh: 0",190.17,190.21,500,200
4,1,1701786600,"PastLimit: 0, Unreported: 0",190.21,183,ARCA,FT,"BidPastLow: 0, AskPastHigh: 0",190.19,190.21,200,200
...,...,...,...,...,...,...,...,...,...,...,...,...
75941,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,1,FINRA,4 I,"BidPastLow: 0, AskPastHigh: 0",190.30,190.32,700,1100
75942,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,1,FINRA,4 I,"BidPastLow: 0, AskPastHigh: 0",190.31,190.32,200,1400
75943,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,1,FINRA,4 I,"BidPastLow: 0, AskPastHigh: 0",190.31,190.33,2500,600
75944,2,1701786602,"PastLimit: 0, Unreported: 1",190.22,1,FINRA,4 I,"BidPastLow: 0, AskPastHigh: 0",190.32,190.33,400,800
